# "A new home"
> "search in progress"

- toc: false
- badges: true
- comments: false
- categories: [altair]

In [78]:
#hide
#- image: images/copied_from_nb/2021-05-01-our-slice-of-paradise.png
import os
from dotenv import load_dotenv
load_dotenv()

True

In [79]:
import os
import requests

import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [80]:
locations = {
    "Karlsruhe": (49.0094, 8.4044, 118),
    "Tenerife":  (28.4682, -16.2546, 55),
    "Lisbon":    (38.7167, -9.1333, 45),
    "Mallorca":  (39.55, 2.7333, 4)
}

base_url = "https://api.meteostat.net/v2/point/climate"
headers = {
    "x-api-key": os.getenv("meteostat_api_key")
}

df = None

for location, coords in locations.items():
    
    lat, lon, elev = coords

    params = {
        "lat": lat,
        "lon": lon,
        "alt": elev
    }

    response = requests.get(base_url, params=params, headers=headers)
    
    _df = pd.DataFrame(response.json()["data"])
    _df["location"] = location
    
    if df is None:
        df = _df
    else:
        df = df.append(_df)

In [81]:
df = df.drop("tsun", axis=1)
df = df.reset_index()

In [82]:
df2 = pd.read_csv("2021-04-29-a-new-home.csv", sep=",", index_col=["location", "month"])

In [83]:
df = df.join(df2[["tsun"]], on=["location", "month"], how="left", sort=True)

In [93]:
filt = (df["location"] != "Mallorca")
df = df[filt]

In [94]:
base_chart = alt.Chart(df)

label = alt.selection_single(
    encodings=["x"], # limit selection to x-axis value
    on="mouseover",  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty="none"     # empty selection includes no data points
)

base = base_chart.mark_line().encode(
        x="month:O",
        y="tavg",
        color="location"
    )

alt.vconcat(

    base_chart.mark_point().encode(
        x="month:O",
        y="tsun",
        color="location"
    ).properties(
        title="Daily Sunshine Hours",
        width=800,
        height=200
    ),

    alt.layer(
        base,
        
        base_chart.mark_area(fillOpacity=.15).encode(
            x="month:O",
            y="tmax",
            y2="tmin",
            color="location"
        ),

            # add a rule mark to serve as a guide line
        base_chart.mark_rule(color="#aaa").encode(
                x="month:O"
            ).transform_filter(label),

        # add circle marks for selected time points, hide unselected points
        base.mark_circle().encode(
                opacity=alt.condition(label, alt.value(1), alt.value(0))
            ).add_selection(label),

        # add white stroked text to provide a legible background for labels
        base.mark_text(align="left", dx=5, dy=-5, stroke="white", strokeWidth=2).encode(
            text="tavg:Q"
        ).transform_filter(label),

        # add text labels for stock prices
        base.mark_text(align="left", dx=5, dy=-5).encode(
            text=alt.Text("tavg:Q")
        ).transform_filter(label),
    ).properties(
        title="Historical Avg/Min/Max Temperatures",
        width=800,
        height=400
    )
)

alt.VConcatChart(...)